In [ ]:
import torch

torch.cuda.set_device(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
from ultralytics import YOLO
from pathlib import Path
import cv2
import os


In [ ]:
from collections import defaultdict
import numpy as np
import pandas as pd

def processVideoTrackYolo(videoName, videoFolder, outputFolder = ''):
    outputCSVFile = outputFolder + videoName + '-output.csv'
    video_path = videoFolder + videoName
    cap = cv2.VideoCapture(video_path)


    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_path = outputFolder + videoName + '_output.mp4'
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    track_history = defaultdict(lambda: [])
    frame_number = 0
    all_detections = []
    
    while cap.isOpened():
        success, frame = cap.read()
    
        if success:
            results = model.track(frame, persist=True, classes = [2, 3, 5, 7])
            frame_number += 1
            if results[0].boxes.id is not None:
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()
        
                annotated_frame = results[0].plot()
        
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]
                    track.append((float(x), float(y)))
                    if len(track) > 30:
                        track.pop(0)
        
                    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                    cv2.polylines(annotated_frame, [points], isClosed=False, color=(255,170,29), thickness=15)
        
                finished_tracks = track_history.keys() - track_ids
                for ft_id in finished_tracks:
                    ft = track_history.pop(ft_id)
                       
                cv2.imshow("YOLOv8 Tracking Vehicles", annotated_frame)

                out.write(annotated_frame)

                detections = results[0].summary(normalize=False, decimals=5)
                
                for detection in detections:
                    detection['frame'] = frame_number
                    detection['x1'] = detection['box']['x1']
                    detection['y1'] = detection['box']['y1']
                    detection['x2'] = detection['box']['x2']
                    detection['y2'] = detection['box']['y2']
                    del detection['box']
                    all_detections.append(detection)
                
            
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            break

    df = pd.DataFrame(all_detections)
    df.to_csv(outputCSVFile, index=False)
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Annotated video saved as {output_path}")

In [ ]:
testVideosFolder = '/media/rv/Disk 0/SST-ComputerVision/Cameras/Retry/check2rd/CrossCamera_cam5_cam8/'
outputVehicleTrack= 'vehicle-track-csv/'

In [ ]:
##For small objects, lower-stride models typically fare better. That's because these models generally maintain more detail from the input image, which can be essential for detecting and correctly classifying small objects. Our YOLOv8s, YOLOv8m, or YOLOv8l models would be great for small objects detection.
##For large objects, higher-stride models often work well. These models effectively sum up larger areas of the image, which can help them detect and classify large objects. In this case, the YOLOv8x and YOLOv8x6 models could be a better option.
##For a mixed-size object detection or segmentation, a model with a mix of strides can often hit the sweet spot. This includes models like YOLOv8l6 or YOLOv8x6, which have additional P6 output with stride 64.

model = YOLO('yolov8x.pt')

In [ ]:


processVideoTrackYolo("M24_cam5_8.avi", testVideosFolder, outputVehicleTrack)

In [ ]:
vehicle_videos = os.listdir(testVideosFolder)

print(vehicle_videos)


In [ ]:
for videoName in vehicle_videos[:2]:
    print('processing: ' + videoName)
    processVideoTrackYolo(videoName, testVideosFolder, outputVehicleTrack)

In [ ]:
from collections import defaultdict
import numpy as np
import pandas as pd
import cv2
from datetime import timedelta

def processVideoTrackYoloTextOnly(videoName, videoFolder, outputFolder = '', save_interval=100):
    outputCSVFile = outputFolder + videoName + '-output.csv'
    cacheCSVFile = outputFolder + videoName + '-cached-output.csv'
    video_path = videoFolder + videoName
    cap = cv2.VideoCapture(video_path)

    frame_number = 0
    all_detections = []
    
    while cap.isOpened():
        success, frame = cap.read()
    
        if success:
            results = model.track(frame, persist=True, classes = [2, 3, 5, 7])
            frame_number += 1
            millis = cap.get(cv2.CAP_PROP_POS_MSEC)
            timestamp = str(timedelta(milliseconds=millis))
            if results[0].boxes.id is not None:
                detections = results[0].summary(normalize=False, decimals=5)
                for detection in detections:
                    detection['frame'] = frame_number
                    detection['timestamp'] = timestamp
                    detection['x1'] = detection['box']['x1']
                    detection['y1'] = detection['box']['y1']
                    detection['x2'] = detection['box']['x2']
                    detection['y2'] = detection['box']['y2']
                    del detection['box']
                    all_detections.append(detection)

            if frame_number % save_interval == 0:
                df = pd.DataFrame(all_detections)
                df.to_csv(cacheCSVFile, index=False)
                print(f"Cached results saved as {cacheCSVFile} at frame {frame_number}")
            
                    
            
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            break

    df = pd.DataFrame(all_detections)
    df.to_csv(outputCSVFile, index=False)
    cap.release()
    print(f"Results csv saved as {outputCSVFile}")

In [ ]:
from collections import defaultdict
import numpy as np
import pandas as pd
import cv2
from datetime import timedelta

def processVideoTrackYoloCSV(videoName, videoFolder, outputFolder = '', save_interval=100):
    outputCSVFile = outputFolder + videoName + '-output.csv'
    cacheCSVFile = outputFolder + videoName + '-cached-output.csv'

    video_path = videoFolder + videoName
    cap = cv2.VideoCapture(video_path)


    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_path = outputFolder + videoName + '_output.mp4'
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    track_history = defaultdict(lambda: [])
    frame_number = 0
    all_detections = []
    
    while cap.isOpened():
        success, frame = cap.read()
    
        if success:
            results = model.track(frame, persist=True, classes = [2, 3, 5, 7])
            frame_number += 1
            if results[0].boxes.id is not None:
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()
        
                annotated_frame = results[0].plot()
        
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]
                    track.append((float(x), float(y)))
                    if len(track) > 30:
                        track.pop(0)
        
                    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                    cv2.polylines(annotated_frame, [points], isClosed=False, color=(255,170,29), thickness=15)
        
                finished_tracks = track_history.keys() - track_ids
                for ft_id in finished_tracks:
                    ft = track_history.pop(ft_id)
                       
                cv2.imshow("YOLOv8 Tracking Vehicles", annotated_frame)

                out.write(annotated_frame)

                detections = results[0].summary(normalize=False, decimals=5)
                millis = cap.get(cv2.CAP_PROP_POS_MSEC)
                timestamp = str(timedelta(milliseconds=millis))
                
                for detection in detections:
                    detection['frame'] = frame_number
                    detection['timestamp'] = timestamp
                    detection['x1'] = detection['box']['x1']
                    detection['y1'] = detection['box']['y1']
                    detection['x2'] = detection['box']['x2']
                    detection['y2'] = detection['box']['y2']
                    del detection['box']
                    all_detections.append(detection)

            if frame_number % save_interval == 0:
                df = pd.DataFrame(all_detections)
                df.to_csv(cacheCSVFile, index=False)
                print(f"Cached results saved as {cacheCSVFile} at frame {frame_number}")
            
                
            
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            break

    df = pd.DataFrame(all_detections)
    df.to_csv(outputCSVFile, index=False)
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Annotated video saved as {output_path}")

In [ ]:
%%time
vehicle_videos = os.listdir(testVideosFolder)

print(vehicle_videos)

for videoName in vehicle_videos[:2]:
    print('processing: ' + videoName)
    processVideoTrackYoloTextOnly(videoName, testVideosFolder, outputVehicleTrack)

In [ ]:
%%time

processVideoTrackYoloTextOnly("M9_cam7_4.avi", testVideosFolder, outputVehicleTrack, save_interval=102)

In [ ]:
orangeVehFolder = '/media/rv/Disk 0/SST-ComputerVision/Cameras/Orange/veh-cams-in-one/'
orange_vehicle_videos = os.listdir(orangeVehFolder)
outputVehicleTrack= 'vehicle-track-csv/'

print(orange_vehicle_videos)

In [ ]:
%%time

for videoName in orange_vehicle_videos:
    print('processing: ' + videoName)
    model = YOLO('yolov8x.pt')
    processVideoTrackYoloTextOnly(videoName, orangeVehFolder, outputVehicleTrack, save_interval=33333)

In [ ]:
orangeVehFolder = '/Users/rockr/OneDrive - University of Central Florida/ucf transportation/code/vehicle-computer-vision-csv/orange/'
orange_vehicle_videos = os.listdir(orangeVehFolder)
outputVehicleTrack= '/Users/rockr/OneDrive - University of Central Florida/ucf transportation/code/vehicle-computer-vision-csv/vehicle-track-csv/'

print(orange_vehicle_videos)

In [ ]:
%%time

for videoName in orange_vehicle_videos:
    print('processing: ' + videoName)
    model = YOLO('yolov8x.pt')
    processVideoTrackYoloCSV(videoName, orangeVehFolder, outputVehicleTrack, save_interval=63333)

In [ ]:
%%time

for videoName in orange_vehicle_videos:
    print('processing: ' + videoName)
    model = YOLO('yolov8x.pt')
    processVideoTrackYoloTextOnly(videoName, orangeVehFolder, outputVehicleTrack, save_interval=63333)

### Optimization

In [ ]:
from video_processing import processVideoTrackYoloCSV

processVideoTrackYoloCSV(videoName, orangeVehFolder, outputVehicleTrack, save_interval=10000)